In [1]:
import scGCO
import time
import pandas as pd
import numpy as np
%load_ext memory_profiler


In [2]:
nc=[]
tt =[]
import time
n_cell=[100,250,500,1000,1500,5000,10000]
for i in n_cell:
    ff='Data/10k_cells_100genes_counts_0.csv'
    data0 = pd.read_csv(ff, sep=',', index_col = 0,iterator=True)
    data=data0.get_chunk(i)
    
    temp = [val.split('x') for val in data.index.values]
    coord = np.array([[float(a[0]), float(a[1])] for a in temp])

    locs=coord
    sub_data=data
    sub_data = sub_data.loc[:,(sub_data != 0).astype(int).sum(axis=0) >= 10]    
    data_norm = scGCO.normalize_count_cellranger(sub_data)

    factor_df, size_factor = scGCO.estimate_smooth_factor(locs, data_norm)

    start = time.time()
    %memit result_df = scGCO.identify_spatial_genes(locs, data_norm, size_factor)
    end = time.time()
    t=end-start
    
    nc.append(int(i))
    tt.append(t)
     

In [4]:
tt

[1.388200044631958,
 2.1948018074035645,
 3.681540012359619,
 6.351463556289673,
 11.330513000488281,
 36.01528239250183,
 65.68832039833069]

In [6]:
nc

[100, 250, 500, 1000, 1500, 5000, 10000]

In [7]:
memory=np.array([144.16 ,146.45,149.16,164.26,181.04,249.53 ,356.57 ])

In [10]:
df=pd.DataFrame({'n.cells':nc,'n.genes':100,'time_s':tt,'memory':memory})

In [11]:
df

,n.cells,n.genes,time_s,memory
0,100,100,1.388200,144.16
1,250,100,2.194802,146.45
2,500,100,3.681540,149.16
3,1000,100,6.351464,164.26
4,1500,100,11.330513,181.04
5,5000,100,36.015282,249.53
6,10000,100,65.688320,356.57
